In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

In [8]:
train = pd.read_csv('train.csv')
train.head()

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0


In [9]:
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('train/'+train['id'][i].astype('str')+'.png', target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

  0%|                                                | 0/60000 [00:00<?, ?it/s]c:\python37\lib\site-packages\keras_preprocessing\image\utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|████████████████████████████████████| 60000/60000 [10:53<00:00, 91.77it/s]


In [11]:
y = train['label'].values
y = to_categorical(y)
y

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [13]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [14]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 382s 8ms/step - loss: 0.5087 - accuracy: 0.8212 - val_loss: 0.3240 - val_accuracy: 0.8826
Epoch 2/10
48000/48000 [==============================] - 383s 8ms/step - loss: 0.3364 - accuracy: 0.8781 - val_loss: 0.2704 - val_accuracy: 0.9022
Epoch 3/10
48000/48000 [==============================] - 368s 8ms/step - loss: 0.2874 - accuracy: 0.8950 - val_loss: 0.2465 - val_accuracy: 0.9093
Epoch 4/10
48000/48000 [==============================] - 396s 8ms/step - loss: 0.2499 - accuracy: 0.9087 - val_loss: 0.2396 - val_accuracy: 0.9104
Epoch 5/10
48000/48000 [==============================] - 350s 7ms/step - loss: 0.2283 - accuracy: 0.9151 - val_loss: 0.2229 - val_accuracy: 0.9182
Epoch 6/10
48000/48000 [==============================] - 330s 7ms/step - loss: 0.2080 - accuracy: 0.9221 - val_loss: 0.2328 - val_accuracy: 0.9165
Epoch 7/10
48000/48000 [==============================] - 318s

In [17]:
test = pd.read_csv('test.csv')

In [18]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('test/'+test['id'][i].astype('str')+'.png', target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

  0%|                                                | 0/10000 [00:00<?, ?it/s]c:\python37\lib\site-packages\keras_preprocessing\image\utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|████████████████████████████████████| 10000/10000 [04:38<00:00, 35.94it/s]


In [19]:
# making predictions
prediction = model.predict_classes(test)

In [20]:
sample = pd.read_csv('sample_submission.csv')
sample['label'] = prediction
sample.to_csv('sample_cnn.csv', header=True, index=False)